<a href="https://colab.research.google.com/github/PatrykUtkala/NLPAlgorithms/blob/main/FastTextUsage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
!pip install fasttext
import fasttext.util
import numpy as np
import itertools
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
ft = fasttext.load_model(r'/content/gdrive/MyDrive/Fasttext/cc.pl.300.bin')
# fasttext.util.reduce_model(ft, 100)

print(ft.get_dimension())

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.6 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3163134 sha256=4ab482e63001383f38b148138dfa46eaad97f3aa0c54953c2a5a010fde2be96f
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


300


In [3]:
!pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.bin.gz
!gzip -d cc.pl.300.bin.gz
import fasttext.util
import numpy as np
import itertools
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
ft = fasttext.load_model(r'cc.pl.300.bin')

print(ft.get_dimension())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.5 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3163177 sha256=f3ecf0f3a0e52b30024797e53c133fb07fa33be6e3172f6b234d89768b0b347c
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
--2022-09-12 07:49:01--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503081312 (4.2G) [application/octet-stream]
Saving to: ‘cc.pl.300.bin.gz’

cc.pl.300.bin.gz    100%[===================>]   4.19G  4

300


In [ ]:
#@title Test
word = 'Paryż'
to_remember = 100
tal = ft.get_word_vector("wampir")
fok = ft.get_word_vector('Paryż')
point = (tal + fok)/2
print(euclidean(point, fok), euclidean(point, tal))
for i in range(20):
  print("nowy:", word)
  a = ft.get_nearest_neighbors(word, k=200000)
  for aa in a:
    if ('Paryż' in aa[1]) or ("wampir" in aa[1]):
      continue
    fok = ft.get_word_vector(aa[1])
    # result = np.sqrt(np.sum((tal - fok)**2))
    result = euclidean(point, fok)
    # print(result, aa[1])
    if result < to_remember:
      to_remember = result
      word = aa[1]
      # print(word, to_remember, np.sqrt(np.sum((fok - ft.get_word_vector('Paryż'))**2)))
      print(word, to_remember,euclidean(ft.get_word_vector(word), ft.get_word_vector('Paryż')))

In [ ]:
words = ['Paryż', 'kura', 'pióra', 'trawnik']
points = [ft.get_word_vector(x) for x in words]
target = np.sum(points, axis=0)/2
word = words[0]
to_remember = 100
for i in range(5):
  print("nowy:", word)
  a = ft.get_nearest_neighbors(word, k=200000)
  for aa in a:
    in_check = False
    for w in words:
      if w in aa[1]:
        in_check=True
    if in_check:
      continue
    fok = ft.get_word_vector(aa[1])
    # result = np.sqrt(np.sum((tal - fok)**2))
    result = euclidean(target, fok)
    # print(result, aa[1])
    if result < to_remember:
      to_remember = result
      word = aa[1]
      # print(word, to_remember, np.sqrt(np.sum((fok - ft.get_word_vector('Paryż'))**2)))
      print(word, to_remember)
      for i, w in enumerate(words):
        print("\t\t", w, euclidean(fok, points[i]))

print("result:", word)


In [18]:
#@title defs
def find_closest(words, number, embeddings):
  if len(words)<number:
    return words
  b = np.array(list(itertools.combinations(words, number)))
  l = []
  for bb in b:
    a = []
    for words in bb:
      a.append(embeddings[words])
    a = np.array(a)
    a = np.array(list(itertools.combinations(a, 2)))
    l.append(np.sum(np.sqrt(np.sum(np.diff(a, axis=1)**2, axis=2))))
  l = np.array(l)
  return b[np.argmin(l)]


def gen_word(words):
  points = [ft.get_word_vector(x) for x in words]
  target = np.sum(points, axis=0)/2
  word = words[0]
  to_remember = 100
  for i in range(5):
    print("nowy:", word)
    a = ft.get_nearest_neighbors(word, k=200000)
    for aa in a:
      in_check = False
      for w in words:
        if w in aa[1]:
          in_check=True
      if in_check:
        continue
      fok = ft.get_word_vector(aa[1])
      # result = np.sqrt(np.sum((tal - fok)**2))
      result = cosine(target, fok)
      # print(result, aa[1])
      if result < to_remember:
        to_remember = result
        word = aa[1]
        # print(word, to_remember, np.sqrt(np.sum((fok - ft.get_word_vector('Paryż'))**2)))
        print(word, to_remember)
        for i, w in enumerate(words):
          print("\t\t", w, cosine(fok, points[i]))

  return(word)

def gen_word_av(words, avoided_words, embs, point):
  points = [embs[x] for x in words]
  target = ft.get_word_vector(point)
  similarity = [cosine(target, x) for x in points]
  sim_max = np.max(similarity)

  anti_similarity = np.array([cosine(target, embs[x]) for x in avoided_words])
  print('test', sim_max, "word:", point)
  print(words)
  print(similarity)
  print(avoided_words)
  print(anti_similarity)
  print(anti_similarity < sim_max)
  
  


##Api

In [3]:
!pip install fastapi nest-asyncio pyngrok uvicorn

from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 745 kB 9.9 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 80 kB 10.0 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=7002e54ed85e6b42f4d229bbad6128963f935acb7c5a4535255d6c2391520f7c
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [19]:
app = FastAPI()

class Map(BaseModel):
    team: int
    current_game_map: list
    current_word_list: list

class Words(BaseModel):
    current_word_list: list
    password: str

@app.get('/index')
async def home():
  return "Hello World"

@app.post('/words')
async def home(my_map: Map):
  team = my_map.team
  my_words = []
  bad_words = []
  for i, t in enumerate(my_map.current_game_map):
    if t == team:
      my_words.append(my_map.current_word_list[i])
    else:
      if my_map.current_word_list[i] != '':
        bad_words.append(my_map.current_word_list[i])
  embeddings = {x: ft.get_word_vector(x) for x in my_map.current_word_list}
  closest = find_closest(my_words, 3, embeddings)

  
  word = gen_word(closest)
  gen_word_av(closest, bad_words, embeddings, word)
  # word = "cwel"
  json_compatible_item_data = jsonable_encoder({"word": word, "number": 3})
  return JSONResponse(content=json_compatible_item_data)

@app.post('/find')
async def home(words: Words):
  password_emb = ft.get_word_vector(words.password)
  embeddings = [ft.get_word_vector(x) for x in words.current_word_list ]
  lowest = 100
  current = None
  for i, emb in enumerate(embeddings):
    if words.current_word_list[i] == '':
      continue
    result = cosine(password_emb, emb)
    print(result, words.current_word_list[i])
    if result < lowest:
      lowest = result
      current = i
      # print(result, words.current_word_list[current])
  print("result", words.current_word_list[current])
  json_compatible_item_data = jsonable_encoder({"word": words.current_word_list[current]})
  return JSONResponse(content=json_compatible_item_data)


  

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://0001-35-243-170-205.ngrok.io


INFO:     Started server process [58]
INFO:uvicorn.error:Started server process [58]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


nowy: kontrakt
Kontrakt 0.5275134742259979
		 kontrakt 0.2128610610961914
		 antarktyka 0.9061001464724541
		 limuzyna 0.9471961408853531
umowa 0.48974525928497314
		 kontrakt 0.47965115308761597
		 antarktyka 0.729116827249527
		 limuzyna 0.7679914683103561
bulwarówka 0.4653586745262146
		 kontrakt 0.767968162894249
		 antarktyka 0.6403999924659729
		 limuzyna 0.5320163071155548
limuzyn 0.4168802499771118
		 kontrakt 0.803046241402626
		 antarktyka 0.8096096962690353
		 limuzyna 0.3009796738624573
limuzyne 0.4031030535697937
		 kontrakt 0.8078867197036743
		 antarktyka 0.7084585130214691
		 limuzyna 0.34261471033096313
nowy: limuzyne
nowy: limuzyne
nowy: limuzyne
nowy: limuzyne
test 0.8078867197036743 word: limuzyne
['kontrakt' 'antarktyka' 'limuzyna']
[0.8078867197036743, 0.7084585130214691, 0.34261471033096313]
['napad', 'moskwa', 'szkło', 'soczewka', 'połączenie', 'ambulans', 'ława', 'przewodnik', 'korzenie', 'talia', 'życie', 'dusza', 'muszla', 'babka', 'żołnierz', 'szpieg']
[0.86

INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Finished server process [58]
INFO:uvicorn.error:Finished server process [58]
